In [175]:
import pandas as pd
import numpy as np
import random
import itertools
import csv

In [51]:
class Card:
    SUIT_MAP = {
        'D': 'Diamonds',
        'S': 'Spades',
        'C': 'Clubs',
        'H': 'Hearts'
    }

    RANK_MAP = {
        '2': "Two",
        '3': "Three",
        '4': "Four",
        '5': "Five",
        '6': "Six",
        '7': "Seven",
        '8': "Eight",
        '9': "Nine",
        '10': "Ten",
        'J': "Jack",
        'Q': "Queen",
        'K': "King",
        'A': "Ace"
    }

    COLOR_MAP = {
        'R': "Red",
        'B': "Black"
    }

    COLOR_SUIT_MAP = {
        'R':['H', 'D'],
        'B': ['S', 'C']
    }

    SUIT_COLOR_MAP = {
        'H': 'R',
        'D': 'R',
        'S': 'B',
        'C': 'B'
    }

    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.color = Card.SUIT_COLOR_MAP[suit]
        self.rank = rank

    def getSuit(self) -> str:
        return self.suit

    def getColor(self) -> str:
        return self.color

    def getRank(self) -> str:
        return self.rank

    def printCard(self):
        name = Card.RANK_MAP[self.rank] + " of " + Card.SUIT_MAP[self.suit]
        print(name)

    def getCard(self):
        return (self.rank + self.suit)

    def getShorthand(self):
        return self.rank + self.suit

    @classmethod
    # TODO fix for 10
    def fromShorthand(cls, shorthand: str):
        suit = shorthand[1]
        rank = shorthand[0]
        return cls(suit,rank)

In [38]:
class Deck:

    SUITS = ['S', 'H', 'D', 'C']
    
    RANKS = {
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '10': 10,
        'J': 11,
        'Q': 12,
        'K': 13,
        'A': 14
    }

    COLORS = ['R','B']

    def __init__(self):
        deck = list()
        for rank in Deck.RANKS:
            for suit in Deck.SUITS:
                deck.append(Card(suit,rank))
        random.shuffle(deck)
        self.deck = deck
        self.size = len(deck)

    def getDeck(self):
        return self.deck

    def drawCards(self, numberOfDraws: int = 1) -> list:

        assert self.size >= numberOfDraws, "Deck doesn't have enough cards!"

        cardsDrawn = []
        for i in range(numberOfDraws):
            cardsDrawn.append(self.deck.pop(-1))
            self.size = self.size - 1
        
        return cardsDrawn

    def shuffleDeck(self):
        random.shuffle(self.deck)



In [13]:
class Hand:

    def __init__(self, deck: Deck, numberOfCards: int = 3):
        self.hand = deck.drawCards(numberOfCards)

    def getHand(self):
        return self.hand
        
    

In [191]:
def getHandRankings():    
    rankings = {}
    
    names = {}

    suits = Deck.SUITS
    ranks = Deck.RANKS
    ranksInv = {val:key for key,val in ranks.items()}

    rank = 1

    existingHands = set()

    # Three of a kind (trail)
    for item in list(ranks.keys())[-1::-1]:
        cards = [(item+suit) for suit in suits]
        members = list(itertools.combinations(cards,3))
        for member in members:
            hand = frozenset(member)
            rankings[hand] = rank
            names[hand] = item+" trail"
            existingHands.add(hand)

        rank+=1

    # Straight flush (pure sequence)
    for item in ['A']:
        for suit in suits:
            cards = [(item2+suit) for item2 in ['A','K','Q']]
            hand = frozenset(cards)
            rankings[hand] = rank
            names[hand] = 'A'+'-'+'K'+'-'+'Q'+" straight flush"
            existingHands.add(hand)

        rank+=1
        for suit in suits:
            cards = [(item2+suit) for item2 in ['A','2','3']]
            hand = frozenset(cards)
            rankings[hand] = rank
            names[hand] = 'A'+'-'+'2'+'-'+'3'+" straight flush"
            existingHands.add(hand)

        rank+=1
    for item in range(13,3,-1):
        for suit in suits:
            cards = [ranksInv[item]+suit, ranksInv[item-1]+suit, ranksInv[item-2]+suit]
            hand = frozenset(cards)
            rankings[hand] = rank
            names[hand] = ranksInv[item]+'-'+ranksInv[item-1]+'-'+ranksInv[item-2]+" straight flush"
            existingHands.add(hand)
        rank+=1


    # Straight (sequence)
    for item in ['A']:
        one = ['A'+suit for suit in suits]
        two = ['K'+suit for suit in suits]
        three = ['Q'+suit for suit in suits]
        full = [one,two,three]
        cards = set(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)

        for hand in cards:
    
            rankings[frozenset(hand)] = rank
            names[hand] = 'A'+'-'+'K'+'-'+'Q'+" straight"
            existingHands.add(frozenset(hand))

        rank+=1


        one = ['A'+suit for suit in suits]
        two = ['2'+suit for suit in suits]
        three = ['3'+suit for suit in suits]
        full = [one,two,three]
        cards = frozenset(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)

        for hand in cards:
            rankings[frozenset(hand)] = rank
            names[hand] = 'A'+'-'+'2'+'-'+'3'+" straight"
            existingHands.add(frozenset(hand))
        rank+=1

    for item in range(13,3,-1):
        one = [ranksInv[item]+suit for suit in suits]
        two = [ranksInv[item-1]+suit for suit in suits]
        three = [ranksInv[item-2]+suit for suit in suits]
        full = [one, two, three]
        cards = frozenset(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)
        
        for hand in cards:
            rankings[frozenset(hand)] = rank
            names[hand] = ranksInv[item]+'-'+ranksInv[item-1]+'-'+ranksInv[item-2]+" straight flush"
            existingHands.add(frozenset(hand))
        rank+=1

    # Flush (color)
    for top in range(14,4, -1):
        for mid in range(top-1, 2, -1):
            if top-mid==1:
                for low in range(mid-2, 1, -1):
                    for suit in suits:
                        cards=[ranksInv[top]+suit, ranksInv[mid]+suit, ranksInv[low]+suit]
                        hand = frozenset(cards)
                        names[hand] = ranksInv[top]+"-top flush"
                        rankings[hand] = rank
                        rank+=1
                        existingHands.add(hand)
            else:
                for low in range(mid-1, 1, -1):
                    if ((top==14 and mid==3) and (low==2)):
                        continue
                    else:
                        for suit in suits:
                            cards=[ranksInv[top]+suit, ranksInv[mid]+suit, ranksInv[low]+suit]
                            hand = frozenset(cards)
                            names[hand] = ranksInv[top]+"-top flush"
                            rankings[hand] = rank
                            rank+=1
                            existingHands.add(hand)

    # Pair
    for pair in range(14,1, -1):
        for top in range(14,1, -1):
            if pair==top:
                continue
            else:
                cards = [(ranksInv[pair]+suit) for suit in suits]
                oneTwo = list(itertools.combinations(cards,2))
                three = [ranksInv[top]+suit for suit in suits]
                full = [oneTwo, three]
                cards = list(itertools.product(*full))
                cards = frozenset([(item[0][0],item[0][1],item[1]) for item in cards])
                for hand in cards:
                    hand = frozenset(hand)
                    rankings[hand] = rank
                    names[hand] = ranksInv[pair]+" pair"
                    rank+=1
                    existingHands.add(hand)

    # No pair (high card)
    for top in range(14,4, -1):
        for mid in range(top-1, 2, -1):
            if top-mid==1:
                for low in range(mid-2, 1, -1):
                    one = [ranksInv[top]+suit for suit in suits]
                    two = [ranksInv[mid]+suit for suit in suits]
                    three = [ranksInv[low]+suit for suit in suits]
                    full = [one,two,three]
                    cards = frozenset(itertools.product(*full))
                    cards = {frozenset(item2) for item2 in cards}
                    cards = cards.difference(existingHands)
                    for hand in cards:
                        hand = frozenset(hand)
                        names[hand] = ranksInv[top]+"-top"
                        rankings[hand] = rank
                        existingHands.add(hand)
                    rank+=1
                        
            else:
                for low in range(mid-1, 1, -1):
                    if ((top==14 and mid==3) and (low==2)):
                        continue
                    else:
                        one = [ranksInv[top]+suit for suit in suits]
                        two = [ranksInv[mid]+suit for suit in suits]
                        three = [ranksInv[low]+suit for suit in suits]
                        full = [one,two,three]
                        cards = frozenset(itertools.product(*full))
                        cards = {frozenset(item2) for item2 in cards}
                        cards = cards.difference(existingHands)
                        for hand in cards:
                            hand = frozenset(hand)
                            rankings[hand] = rank
                            names[hand] = ranksInv[top]+"-top"
                            existingHands.add(hand)
                        rank+=1

    return rankings,names


In [39]:
# TODO demonstration

deck = Deck()

print("Drawing 20 cards")
drawn = []
left = []
for item in deck.drawCards(20):
    drawn.append(item.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards drawn:", drawn)
print("Deck remaining:", left)
print()

print("Drawing a hand of 3 cards")
hand = Hand(deck)
handCards = []
left = []
for card in hand.getHand():
    handCards.append(card.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards in hand:", handCards)
print("Deck remaining:", left)
print()

Drawing 20 cards
Deck size remaining 32
Cards drawn: ['10D', '10S', '7S', 'KH', '3S', 'QS', '8H', '2D', '3C', 'AH', 'AC', '7C', '2S', 'JC', '3H', '8S', '9C', '7D', '7H', 'AS']
Deck remaining: ['QD', '9S', 'QC', '4D', '10H', '4C', 'JH', '4H', '3D', 'QH', '8C', '6C', 'KD', '5C', 'AD', '5D', '5H', '9D', '10C', '6D', 'JS', '2C', '6H', '4S', 'KC', '9H', '8D', '6S', '5S', 'KS', 'JD', '2H']

Drawing a hand of 3 cards
Deck size remaining 29
Cards in hand: ['2H', 'JD', 'KS']
Deck remaining: ['QD', '9S', 'QC', '4D', '10H', '4C', 'JH', '4H', '3D', 'QH', '8C', '6C', 'KD', '5C', 'AD', '5D', '5H', '9D', '10C', '6D', 'JS', '2C', '6H', '4S', 'KC', '9H', '8D', '6S', '5S']



In [185]:
class Game:

    HAND_RANKINGS, HAND_NAMES = getHandRankings()

    def __init__(self, noOfPlayers: int, jokers: list = None, handSize: int = 3):
        self.noOfPlayers = noOfPlayers
        self.deck = Deck()
        self.jokers = jokers
        self.handSize = handSize
        self.hands = self.generateHands()
        #self.winner = None
        #self.losers = None
        #self.tie = None

    def generateHands(self):
        self.deck.shuffleDeck()
        hands = []
        handsShorthand = []
        for _ in range(self.noOfPlayers):
            hands.append([])
        for _ in range(self.handSize):
            for i in range(self.noOfPlayers):
                hands[i].append(self.deck.drawCards(1)[0])

        for i in range(self.noOfPlayers):
            hands[i] = frozenset([item.getShorthand() for item in hands[i]])

        return hands

    def simulateGame(self):
        results = {item:np.array([0,0,1]) for item in self.hands}
        ranks = np.array([Game.HAND_RANKINGS[hand] for hand in self.hands])
        minIndices = np.where(ranks == ranks.min())[0]
        if len(minIndices)==1:
            win=1
            tie=0
        else:
            win=0
            tie=1
        for item in minIndices:
            results[self.hands[item]]=np.array([win,tie,1])
        return results


In [181]:
simulations2 = {item:np.array([0,0,0]) for item in Game.HAND_RANKINGS.keys()}

for i in range(10000000):
    game = Game(4)
    res = game.simulateGame()
    for item in res:
        simulations2[item]=np.add(simulations2[item],res[item])

In [183]:
for item in simulations2:
    print(item,simulations2[item],round(simulations2[item][0]/simulations2[item][2],3))

frozenset({'AH', 'AD', 'AS'}) [1767    0 1767] 1.0
frozenset({'AH', 'AC', 'AS'}) [1789    0 1789] 1.0
frozenset({'AC', 'AD', 'AS'}) [1859    0 1859] 1.0
frozenset({'AH', 'AC', 'AD'}) [1808    0 1808] 1.0
frozenset({'KS', 'KH', 'KD'}) [1773    0 1774] 0.999
frozenset({'KS', 'KH', 'KC'}) [1803    0 1803] 1.0
frozenset({'KS', 'KC', 'KD'}) [1818    0 1820] 0.999
frozenset({'KH', 'KC', 'KD'}) [1796    0 1799] 0.998
frozenset({'QS', 'QD', 'QH'}) [1791    0 1796] 0.997
frozenset({'QS', 'QC', 'QH'}) [1780    0 1782] 0.999
frozenset({'QS', 'QD', 'QC'}) [1839    0 1840] 0.999
frozenset({'QD', 'QC', 'QH'}) [1767    0 1769] 0.999
frozenset({'JS', 'JD', 'JH'}) [1765    0 1770] 0.997
frozenset({'JS', 'JH', 'JC'}) [1825    0 1829] 0.998
frozenset({'JS', 'JD', 'JC'}) [1815    0 1819] 0.998
frozenset({'JD', 'JH', 'JC'}) [1810    0 1810] 1.0
frozenset({'10S', '10D', '10H'}) [1832    0 1837] 0.997
frozenset({'10S', '10C', '10H'}) [1854    0 1857] 0.998
frozenset({'10S', '10C', '10D'}) [1817    0 1820] 0.

In [177]:
with open('winrate_8_raw.csv', 'w') as csvfile:
    for key in simulations.keys():
        csvfile.write("%s,%s\n"%(key,simulations[key]))

In [229]:
cleaned = pd.DataFrame.from_dict(simulations,orient='index', columns = ['wins','ties','total_games'])
cleaned.index.name = 'hand'
cleaned.insert(0, 'name', list(names.values()))
cleaned['winrate'] = cleaned['wins']/cleaned['total_games']
cleaned['winrate'] = round(cleaned['winrate'], 4)
cleaned.index = [", ".join(list(item)) for item in cleaned.index]


In [230]:
cleaned

,name,wins,ties,total_games,winrate
"AH, AD, AS",A trail,1840,0,1840,1.0000
"AH, AC, AS",A trail,1831,0,1831,1.0000
"AC, AD, AS",A trail,1823,0,1823,1.0000
"AH, AC, AD",A trail,1814,0,1814,1.0000
"KS, KH, KD",K trail,1897,0,1898,0.9995
...,...,...,...,...,...
"3C, 2H, 5D",5-top,0,0,1802,0.0000
"3D, 5H, 2C",5-top,0,0,1874,0.0000
"5S, 3D, 2H",5-top,0,0,1796,0.0000
"3D, 5C, 2C",5-top,0,0,1806,0.0000


In [231]:
cleaned = pd.DataFrame.from_dict(simulations2,orient='index', columns = ['wins','ties','total_games'])
cleaned.index.name = 'hand'
cleaned.insert(0, 'name', list(names.values()))
cleaned['winrate'] = cleaned['wins']/cleaned['total_games']
cleaned['winrate'] = round(cleaned['winrate'], 4)
cleaned.index = [", ".join(list(item)) for item in cleaned.index]

In [232]:
cleaned

,name,wins,ties,total_games,winrate
"AH, AD, AS",A trail,1767,0,1767,1.0000
"AH, AC, AS",A trail,1789,0,1789,1.0000
"AC, AD, AS",A trail,1859,0,1859,1.0000
"AH, AC, AD",A trail,1808,0,1808,1.0000
"KS, KH, KD",K trail,1773,0,1774,0.9994
...,...,...,...,...,...
"3C, 2H, 5D",5-top,0,0,1792,0.0000
"3D, 5H, 2C",5-top,0,0,1831,0.0000
"5S, 3D, 2H",5-top,0,0,1830,0.0000
"3D, 5C, 2C",5-top,0,0,1769,0.0000


In [217]:
cleaned.to_csv('winrate_4_cleaned.csv')

In [193]:
for item in names:
    print(item, names[item])

frozenset({'AH', 'AD', 'AS'}) A trail
frozenset({'AH', 'AC', 'AS'}) A trail
frozenset({'AC', 'AD', 'AS'}) A trail
frozenset({'AH', 'AC', 'AD'}) A trail
frozenset({'KS', 'KH', 'KD'}) K trail
frozenset({'KS', 'KH', 'KC'}) K trail
frozenset({'KS', 'KC', 'KD'}) K trail
frozenset({'KH', 'KC', 'KD'}) K trail
frozenset({'QS', 'QD', 'QH'}) Q trail
frozenset({'QS', 'QC', 'QH'}) Q trail
frozenset({'QS', 'QD', 'QC'}) Q trail
frozenset({'QD', 'QC', 'QH'}) Q trail
frozenset({'JS', 'JD', 'JH'}) J trail
frozenset({'JS', 'JH', 'JC'}) J trail
frozenset({'JS', 'JD', 'JC'}) J trail
frozenset({'JD', 'JH', 'JC'}) J trail
frozenset({'10S', '10D', '10H'}) 10 trail
frozenset({'10S', '10C', '10H'}) 10 trail
frozenset({'10S', '10C', '10D'}) 10 trail
frozenset({'10C', '10D', '10H'}) 10 trail
frozenset({'9S', '9H', '9D'}) 9 trail
frozenset({'9S', '9C', '9H'}) 9 trail
frozenset({'9S', '9C', '9D'}) 9 trail
frozenset({'9C', '9H', '9D'}) 9 trail
frozenset({'8H', '8D', '8S'}) 8 trail
frozenset({'8H', '8S', '8C'}) 8 tr

In [258]:
from simulation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [238]:
load = saveRaw(simulations,8)

In [249]:
res = loadRaw(8)

a


In [254]:
resClean = clean(res, 8)

In [256]:
resClean

,name,wins,ties,total_games,winrate
"AH, AD, AS",A trail,1840,0,1840,1.0000
"AH, AC, AS",A trail,1831,0,1831,1.0000
"AC, AD, AS",A trail,1823,0,1823,1.0000
"AH, AC, AD",A trail,1814,0,1814,1.0000
"KS, KH, KD",K trail,1897,0,1898,0.9995
...,...,...,...,...,...
"3C, 2H, 5D",5-top,0,0,1802,0.0000
"3D, 5H, 2C",5-top,0,0,1874,0.0000
"5S, 3D, 2H",5-top,0,0,1796,0.0000
"3D, 5C, 2C",5-top,0,0,1806,0.0000


In [257]:
saveCleaned(resClean, 8)

In [261]:
a = runSimulations(5, 100)

  0%|          | 0/100 [00:00<?, ?it/s]